In [1]:
import pandas as pd

In [101]:
import re

In [495]:
rel_train = pd.read_csv('relevance_train.csv')
rel_test = pd.read_csv('relevance_test.csv')

In [496]:
regex = re.compile('[^a-zA-Z]')
#First parameter is the replacement, second parameter is your input string
regex.sub('', 'ab3d*E')

'abdE'

In [502]:
queries = {}
with open('queries.csv') as file:
    lines = file.readlines()
    for line in lines:
        if '\t' in line:
            splits = line.split('\t')
            t = regex.sub(' ', splits[1])
            queries[splits[0]] = clean_query(t)

In [503]:
article_start = 0
article_lines = []

query2text = {}

lines = []
with open('Documents.csv') as docs:
    lines = docs.readlines()

for i,line in enumerate(lines):
    if re.match('.*Id.*\d', line) is not None and article_start != i:
        article_lines = lines[article_start:i]
        article_start = i
        q_id = line.split()[1]
        query2text[q_id] = parse_article_lines(article_lines)

In [293]:
from functools import reduce
import nltk

In [294]:
ps = nltk.stem.snowball.SnowballStemmer('english')

In [295]:
import stop_words

In [296]:
sw = stop_words.get_stop_words('english')

In [501]:
def clean_query(tq):
    result = " "
    for w in tq.split():
        if w not in sw:
            result += ps.stem(w.replace(r'[^a-zA-Z]', '')) + " "
    return result

In [500]:
def parse_article_lines(lines):
    text_index = 0
    for i,l in enumerate(lines):
        if '.W' in l:
            text_index = i+1
    
    text_lines = lines[text_index:len(lines)]
    text_lines = list(map(lambda l: regex.sub(' ',l), text_lines))
    text_lines = list(map(clean_query, text_lines))
    return text_lines

In [475]:
def flat(list_of_list):
    result = []
    for li in list_of_list:
        for el in li:
            result.append(el)
            
    result = list(filter(lambda l:len(line) > 1, result))
    
    return result

In [504]:
for k in query2text.keys():
    query2text[k] = flat(list(map(lambda l: l.split(), query2text[k])))

In [505]:
for k in query2text.keys():
    query2text[k] = list(filter(lambda l: len(l) > 2, query2text[k]))

In [506]:
for k in queries.keys():
    queries[k] = queries[k].split()

In [507]:
for k in queries.keys():
    queries[k] = list(filter(lambda l:len(line) > 2, queries[k]))

In [121]:
import gensim

In [480]:
all_w = list(query2text.values())
all_w.append(queries.values())

In [353]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [513]:
model = gensim.models.Word2Vec(min_count=1,workers=4, hs=1, negative=0)

In [514]:
model.build_vocab(flat(all_w))

In [515]:
kw = model.wv

In [516]:
kw.n_similarity(query2text['5'], queries['2'])

KeyError: "word 'dimension' not in vocabulary"